# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [15]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [3]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [4]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [6]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

78693762929874600000000000

## Use this to import default tbears wallet and run the transactions


In [14]:
password = "test1_Account"
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
print(deployer_wallet.get_address())
icon_service.get_balance(deployer_wallet.get_address())

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb


786640639999000000000000000

# Deploying all contracts


In [91]:



params={}

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content("delegation"))\
    .params(params)\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)


contracts['delegations']=res['scoreAddress']


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x4a7e19816d609ad44192cf96cb2ab04485d1c7e656ba66152b916a035bb4a83f', 'blockHeight': 12519438, 'blockHash': '0xfdb7dc6e9febb3b5dcae2ed554b5a73ce0c1f9b46244f5fca0b5da9b2b992224', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxfa6e2afdb3356d023820aaa4f3c1e82d89e7afff', 'stepUsed': 1090580000, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1090580000, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [92]:
pprint(contracts)

{'delegation': 'cx63f0b47ed25a7db4fe5812e00dfa640ea0e31f55',
 'delegations': 'cxfa6e2afdb3356d023820aaa4f3c1e82d89e7afff'}


In [89]:
contracts ={}
contracts['delegation']="cx63f0b47ed25a7db4fe5812e00dfa640ea0e31f55"

## Updating the contract

In [97]:

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['delegations'])\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content("delegation"))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xd1cb4a14fe46e1bf18781b70ae2bf0674f46c73eac58d427f638c2a75d1730b6',
 'blockHeight': 12519688,
 'blockHash': '0xfb91d95c68a26c455f1dc6cb0753cb536557ffc36fc019859320e07226344c85',
 'txIndex': 1,
 'to': 'cxfa6e2afdb3356d023820aaa4f3c1e82d89e7afff',
 'scoreAddress': 'cxfa6e2afdb3356d023820aaa4f3c1e82d89e7afff',
 'stepUsed': 1692981600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1692981600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding delegation



In [95]:

params = { "_delegations":
                
                  [{"prepAddress":"cx9a5a9c116379ecb9e4aadb423955fc9351771aa6","prepPercentage":"400000000000000000"}]
                 
                
          }

# def update_bal_token_dist_percentage(self, _data_source_dict_list : List[DistPercentDict])
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['delegations']) \
    .nid(NID) \
    .nonce(100) \
    .method("updateDelegations")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)


tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32005, 'message': 'Cannot create an interface SCORE with a None address'}


JSONRPCException: {'code': -32005, 'message': 'Cannot create an interface SCORE with a None address'} (Json rpc error)

## Adding New Data Source

In [11]:
params = {'_user': "hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['delegation'])\
    .method("getDelegations")\
    .params(params)\
    .build()
response = icon_service.call(_call)
# for key,value in response.items():
#     if key == "balTokenDistPercent":
#         response[key]=int(response[key],0)/ 10**18


print(response)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32000, 'message': 'Server error'}


JSONRPCException: {'code': -32000, 'message': 'Server error'} (Json rpc error)

## Distribute

In [ ]:

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['rewards']) \
    .nid(NID) \
    .nonce(100) \
    .method("distribute")\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)


tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Get Data Source

In [53]:
params = {'_data_source_name': "Omm1"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['rewards'])\
    .method("get_data_sources")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,value in response.items():
    if key == "balTokenDistPercent":
        response[key]=int(response[key],0)/ 10**18


print(response)

{'day': '0x0', 'contract_address': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37', 'balTokenDistPercent': 0.4}


## Get Bal Token Dist Per Day

In [30]:
# def bal_token_dist_per_day(self, _day: int) -> int:

params = {'_day': 1}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['rewards'])\
    .method("bal_token_dist_per_day")\
    .params(params)\
    .build()
response = icon_service.call(_call)

response=int(response,0) / 10**18
print(response)
        

100000.0
